# ASPECT BASED SENTIMENT ANALYSIS SYSTEM FOR RESTAURANT REVIEW


# Importing Libraries

In [21]:
#Import necessary libraries
import xml.etree.ElementTree as ET
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
import tkinter as tk
from tkinter import ttk
import numpy as np

# Backend code

**Function to Parse XML Data**

In [ ]:
# Function to parse the XML data
def parse_xml(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()

    data = []
    for review in root.findall('Review'):
        rid = review.get('rid')
        sentences = []
        for sentence in review.findall('sentences/sentence'):
            sid = sentence.get('id')
            text = sentence.find('text').text
            opinions = []
            for opinion in sentence.findall('Opinions/Opinion'):
                target = opinion.get('target')
                category = opinion.get('category')
                polarity = opinion.get('polarity')
                entity, attribute = category.split('#')
                opinions.append((entity, attribute, polarity))
            sentences.append((sid, text, opinions))
        data.append((rid, sentences))
    return data

**Preprocessing and Training the Data** 

In [ ]:
# Function to preprocess text
def preprocess_text(text):
    tokens = word_tokenize(text.lower())
    stemmer = PorterStemmer()
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)

# Load and preprocess the training data
train_data_path = 'ABSA_TRAIN.xml'
train_data = parse_xml(train_data_path)

X_train = []
y_train = []
for rid, sentences in train_data:
    for sid, text, opinions in sentences:
        for entity, attribute, polarity in opinions:
            processed_text = preprocess_text(text)
            X_train.append(processed_text)
            y_train.append(polarity)

# Create TF-IDF vectorizer and transform data
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(X_train)

# Update labels to include 'neutral'
labels = ['negative', 'neutral', 'positive']

# Train a Logistic Regression model
model = LogisticRegression(multi_class='multinomial', solver='lbfgs')
model.fit(X_train_tfidf, y_train)

**Function to Predict Polarity**

In [ ]:
# Function to predict polarity for a new review with confidence threshold
def predict_polarity(text, threshold=0.2):
    processed_text = preprocess_text(text)
    text_tfidf = vectorizer.transform([processed_text])
    probabilities = model.predict_proba(text_tfidf)[0]
    
    # Check if difference in probabilities between highest and second highest probabilities is below threshold
    diff = np.max(probabilities) - np.sort(probabilities)[-2]
    if diff < threshold:
        return 'neutral'
    
    predicted_polarity = model.predict(text_tfidf)[0]
    return predicted_polarity

# Function to get the emoji and color based on polarity
def get_emoji_and_color(polarity):
    if polarity == 'negative':
        emoji = '😞'
        color = '#ff4d4d'  # Red
    elif polarity == 'neutral':
        emoji = '😐'
        color = '#ffff00'  # Yellow
    else:
        emoji = '😃'
        color = '#00ff00'  # Green
    return emoji, color

# Frontend Code

**GUI Creation**

In [23]:

def predict_and_display():
    review_text = input_text.get("1.0", "end-1c")  # Get the review text from the input field
    predicted_polarity = predict_polarity(review_text)  # Call backend function to predict polarity
    emoji, color = get_emoji_and_color(predicted_polarity)  # Call backend function to get emoji and color
    
    # Update the output label with predicted polarity, emoji, and color
    output_label.config(text=f"Predicted Polarity: {predicted_polarity} {emoji}", foreground=color)

# Create GUI
root = tk.Tk()
root.title("Sentiment Analysis Recommendation System")

# Create a label for the input field
input_label = ttk.Label(root, text="Input any reviews:")
input_label.pack(pady=5)

# Create a text box for inputting reviews
input_text = tk.Text(root, height=5, width=50)
input_text.pack(pady=5)

# Create a button to predict the polarity
predict_button = ttk.Button(root, text="Predict Polarity", command=predict_and_display)
predict_button.pack(pady=5)

# Create a label to display the predicted polarity and emoji
output_label = ttk.Label(root, text="", font=("Arial", 24))
output_label.pack(pady=5)

root.mainloop()